In [1]:
import pandas as pd
import random 
import numpy as np

## Data Cleaning

In [2]:
set3 = pd.read_csv('subset-3-sous-ensemble-3.csv', encoding='ISO-8859-1')
# print(set3[set3.LEVEL2ID!=0])
set3.drop(columns=['LEVEL2ID', 'LEVEL3ID','LEVEL4ID','LEVEL5ID','DESCRIP_F','TITLE_F','DEPT_F', 'INDICATORFRA','SUBINDICATORFRA'],inplace=True)
set3.head()

,LEVEL1ID,SURVEYR,BYCOND,DESCRIP_E,DEMCODE,QUESTION,TITLE_E,ANSWER1,ANSWER2,ANSWER3,...,MOST_NEGATIVE_OR_LEAST_POSITIVE,AGREE,SCORE5,SCORE100,ANSCOUNT,DEPT_E,INDICATORID,INDICATORENG,SUBINDICATORID,SUBINDICATORENG
0,0,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",35,45,6,...,14,81,3.99,75,73497,Public Service,4,WORKPLACE,14,Physical environment and equipment
1,1,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",41,45,5,...,10,85,4.14,79,11550,Canada Revenue Agency,4,WORKPLACE,14,Physical environment and equipment
2,2,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",45,42,4,...,9,87,4.21,80,5626,Employment and Social Development Canada,4,WORKPLACE,14,Physical environment and equipment
3,3,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",29,48,6,...,16,78,3.86,72,7308,Department of National Defence,4,WORKPLACE,14,Physical environment and equipment
4,4,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",20,46,10,...,23,67,3.58,64,3252,Correctional Service Canada,4,WORKPLACE,14,Physical environment and equipment


## Lots of rows doesn't have survey data (more than 50%)

In [3]:
print(len(set3[set3['ANSWER1']== ' '].index))
print(len(set3[set3['ANSWER5']== ' '].index))
print(len(set3[set3['ANSWER2']!= ' '].index))

# clean out empty rows
print(len(set3))
set3 = set3[set3['ANSWER2']!= ' ']
set3.reset_index(inplace=True, drop=True)
# set3_2020.to_csv('subset3_2020_cleaned.csv')
set3.head()

822456
822456
583123
1405579


,LEVEL1ID,SURVEYR,BYCOND,DESCRIP_E,DEMCODE,QUESTION,TITLE_E,ANSWER1,ANSWER2,ANSWER3,...,MOST_NEGATIVE_OR_LEAST_POSITIVE,AGREE,SCORE5,SCORE100,ANSCOUNT,DEPT_E,INDICATORID,INDICATORENG,SUBINDICATORID,SUBINDICATORENG
0,0,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",35,45,6,...,14,81,3.99,75,73497,Public Service,4,WORKPLACE,14,Physical environment and equipment
1,1,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",41,45,5,...,10,85,4.14,79,11550,Canada Revenue Agency,4,WORKPLACE,14,Physical environment and equipment
2,2,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",45,42,4,...,9,87,4.21,80,5626,Employment and Social Development Canada,4,WORKPLACE,14,Physical environment and equipment
3,3,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",29,48,6,...,16,78,3.86,72,7308,Department of National Defence,4,WORKPLACE,14,Physical environment and equipment
4,4,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",20,46,10,...,23,67,3.58,64,3252,Correctional Service Canada,4,WORKPLACE,14,Physical environment and equipment


## Take year 2020 data

In [8]:
print(set3.columns)
print(set3.SURVEYR.unique())
print(len(set3[set3.SURVEYR==2020]))
# 2020 has 330k rows
# 2019 has 176k rows, 
# 2018 has 76k rows

# have majority of data form 2020 so we focus on this year
set3_2020 = set3[set3.SURVEYR==2020]

Index(['LEVEL1ID', 'SURVEYR', 'BYCOND', 'DESCRIP_E', 'DEMCODE', 'QUESTION',
       'TITLE_E', 'ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5',
       'ANSWER6', 'ANSWER7', 'MOST_POSITIVE_OR_LEAST_NEGATIVE',
       'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE',
       'AGREE', 'SCORE5', 'SCORE100', 'ANSCOUNT', 'DEPT_E', 'INDICATORID',
       'INDICATORENG', 'SUBINDICATORID', 'SUBINDICATORENG'],
      dtype='object')
[2020 2019 2018]
330739


In [24]:
for i in set3_2020.groupby('DEPT_E').size():
    print(i)
set3_2020.groupby('DEPT_E').size()

2273
6978
2574
9073
1924
3107
10505
2862
6774
2640
5751
2205
2343
3406
2843
348
2706
1887
5458
9708
2971
5349
2461
6937
9602
10456
7622
1416
1612
8278
8063
7977
4864
7974
2720
7629
2772
6876
2638
4450
1634
6097
6501
2623
2567
3292
1482
1384
6699
3371
6048
3910
4755
12217
4151
9055
7690
7519
2099
5397
7132
1595
7530
1263
6292
6254
2314
1836


DEPT_E
Administrative Tribunals Support Service of Canada    2273
Agriculture and Agri-Food Canada                      6978
Atlantic Canada Opportunities Agency                  2574
Canada Border Services Agency                         9073
Canada Economic Development for Quebec Regions        1924
                                                      ... 
Transportation Safety Board of Canada                 1263
Treasury Board of Canada Secretariat                  6292
Veterans Affairs Canada                               6254
Western Economic Diversification Canada               2314
Women and Gender Equality Canada                      1836
Length: 68, dtype: int64

In [11]:
dept = set3_2020.groupby('DEPT_E').groups
for d, info in dept.items():
    print(d)

Administrative Tribunals Support Service of Canada
Agriculture and Agri-Food Canada
Atlantic Canada Opportunities Agency
Canada Border Services Agency
Canada Economic Development for Quebec Regions
Canada Energy Regulator
Canada Revenue Agency
Canada School of Public Service
Canadian Food Inspection Agency
Canadian Grain Commission
Canadian Heritage
Canadian Human Rights Commission
Canadian Institutes of Health Research
Canadian Nuclear Safety Commission
Canadian Radio-television and TeleCommunications Commission
Canadian Security Intelligence Service
Canadian Space Agency
Canadian Transportation Agency
Communications Security Establishment Canada
Correctional Service Canada
Courts Administration Service
Crown-Indigenous Relations and Northern Affairs Canada
Department of Finance Canada
Department of Justice
Department of National Defence
Employment and Social Development Canada
Environment and Climate Change Canada
Federal Economic Development Agency for Southern Ontario
Financial Con

822456
822456


In [139]:
# set3[].head()

b1 = list(set3['ANSWER1']== ' ')
b2 = list(set3['DEPT_E']=='Fisheries and Oceans Canada')
b3 = b1 and b2
# print(len(b1))
# print(len(b3))
# set3[b3]
set3[b3]

,LEVEL1ID,SURVEYR,BYCOND,DESCRIP_E,DESCRIP_F,DEMCODE,QUESTION,TITLE_E,TITLE_F,ANSWER1,...,SCORE100,ANSCOUNT,DEPT_E,DEPT_F,INDICATORID,INDICATORENG,INDICATORFRA,SUBINDICATORID,SUBINDICATORENG,SUBINDICATORFRA
id,,,,,,,,,,,,,,,,,,,,,
5,5,2020,Q115 = 1,Male gender,Genre masculin,2011,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",31,...,73,3010,Fisheries and Oceans Canada,Pêches et Océans Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
73,5,2020,Q115 = 2,Female gender,Genre féminin,2012,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",37,...,77,3443,Fisheries and Oceans Canada,Pêches et Océans Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
141,5,2020,Q115 = 3,Gender diverse,Genre divers,2013,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",,...,,,Fisheries and Oceans Canada,Pêches et Océans Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
192,5,2020,Q116 = 1,Indigenous,Autochtone,2014,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",28,...,72,320,Fisheries and Oceans Canada,Pêches et Océans Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
259,5,2020,Q116 = 2,Non-Indigenous,Non-Autochtone,2015,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",34,...,75,6195,Fisheries and Oceans Canada,Pêches et Océans Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405276,5,2019,Q122 = 4,Another sexual orientation,Une autre orientation sexuelle,2068,Q97,Question 97. To what extent have issues with t...,Question 97. Dans quelle mesure les problèmes ...,42,...,67,22,Fisheries and Oceans Canada,Pêches et Océans Canada,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1405333,5,2018,Q122 = 4,Another sexual orientation,Une autre orientation sexuelle,2068,Q97,Question 97. To what extent have issues with t...,Question 97. Dans quelle mesure les problèmes ...,36,...,62,42,Fisheries and Oceans Canada,Pêches et Océans Canada,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1405389,5,2020,Q122 = 5,Prefer not to answer,Préfère ne pas répondre,2069,Q97,Question 97. To what extent have issues with t...,Question 97. Dans quelle mesure les problèmes ...,36,...,67,1033,Fisheries and Oceans Canada,Pêches et Océans Canada,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération


In [89]:
# set3['DESCRIP_ID'] = 0
# DESCRIP_E = list(set3.DESCRIP_E.unique())
# id_ = 0
# for equity_group in DESCRIP_E:
# #     set3[set3['DESCRIP_E']==equity_group]['DESCRIP_ID'] = id_
#     set3.loc[equity_group]['DESCRIP_ID'] = id_
#     id_+=1
# set3.head()

In [90]:
DESCRIP_E = list(set3.DESCRIP_E.unique())
random_group = DESCRIP_E[random.randint(0,len(DESCRIP_E)-1)]
print(random_group)

Inuk (Inuit)


In [91]:
rand_group_df =  set3.loc[:,'DESCRIP_E']==random_group
set3[rand_group_df]

,LEVEL1ID,LEVEL2ID,LEVEL3ID,LEVEL4ID,LEVEL5ID,SURVEYR,BYCOND,DESCRIP_E,DESCRIP_F,DEMCODE,...,SCORE100,ANSCOUNT,DEPT_E,DEPT_F,INDICATORID,INDICATORENG,INDICATORFRA,SUBINDICATORID,SUBINDICATORENG,SUBINDICATORFRA
452,0,0,0,0,0,2020,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,77,248,Public Service,Fonction publique,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
453,1,0,0,0,0,2020,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,,,Canada Revenue Agency,Agence du revenu du Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
454,2,0,0,0,0,2020,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,80,25,Employment and Social Development Canada,Emploi et Développement social Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
455,3,0,0,0,0,2020,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,80,15,Department of National Defence,Ministère de la Défense nationale,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
456,4,0,0,0,0,2020,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,66,13,Correctional Service Canada,Service correctionnel Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399760,83,0,0,0,0,2019,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,,,Canada Border Services Agency,Agence des services frontaliers du Canada,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1399761,86,0,0,0,0,2019,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,,,Canadian Food Inspection Agency,Agence canadienne d'inspection des aliments,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1399762,88,0,0,0,0,2019,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,71,36,Parks Canada,Parcs Canada,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1399763,94,0,0,0,0,2019,Q117 = 3,Inuk (Inuit),Inuk (Inuit),2018,...,,,Statistical Survey Operations,Opérations des enquêtes statistiques,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération


In [93]:
set3

,LEVEL1ID,SURVEYR,BYCOND,DESCRIP_E,DESCRIP_F,DEMCODE,QUESTION,TITLE_E,TITLE_F,ANSWER1,...,SCORE100,ANSCOUNT,DEPT_E,DEPT_F,INDICATORID,INDICATORENG,INDICATORFRA,SUBINDICATORID,SUBINDICATORENG,SUBINDICATORFRA
0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",35,...,75,73497,Public Service,Fonction publique,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
1,1,2020,Q115 = 1,Male gender,Genre masculin,2011,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",41,...,79,11550,Canada Revenue Agency,Agence du revenu du Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
2,2,2020,Q115 = 1,Male gender,Genre masculin,2011,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",45,...,80,5626,Employment and Social Development Canada,Emploi et Développement social Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
3,3,2020,Q115 = 1,Male gender,Genre masculin,2011,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",29,...,72,7308,Department of National Defence,Ministère de la Défense nationale,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
4,4,2020,Q115 = 1,Male gender,Genre masculin,2011,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn...",20,...,64,3252,Correctional Service Canada,Service correctionnel Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405574,91,2018,Q122 = 5,Prefer not to answer,Préfère ne pas répondre,2069,Q97,Question 97. To what extent have issues with t...,Question 97. Dans quelle mesure les problèmes ...,61,...,82,56,Administrative Tribunals Support Service of Ca...,Service canadien d'appui aux tribunaux adminis...,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1405575,92,2018,Q122 = 5,Prefer not to answer,Préfère ne pas répondre,2069,Q97,Question 97. To what extent have issues with t...,Question 97. Dans quelle mesure les problèmes ...,65,...,81,35,National Film Board of Canada,Office national du film du Canada,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1405576,93,2018,Q122 = 5,Prefer not to answer,Préfère ne pas répondre,2069,Q97,Question 97. To what extent have issues with t...,Question 97. Dans quelle mesure les problèmes ...,51,...,82,33,Federal Economic Development Agency for Southe...,Agence fédérale de développement économique po...,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
1405577,94,2018,Q122 = 5,Prefer not to answer,Préfère ne pas répondre,2069,Q97,Question 97. To what extent have issues with t...,Question 97. Dans quelle mesure les problèmes ...,36,...,80,39,Statistical Survey Operations,Opérations des enquêtes statistiques,6,COMPENSATION,RÉMUNÉRATION,22,Pay or other compensation issues,Problèmes de paye ou de rémunération
